# CNN  (Mnist)

Kerasを使うとレイヤーのように積み重なるだけで作れるが，tensorflowにも同じような機能があるので紹介   
(tensorflowにKerasのレイヤーを取り込んで言ってるって聞いたことも...)

必要なものをinport する

In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle


tensorflowにあるデータセットを読み込む

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tmp/data/", one_hot=True)

学習データの削減+扱いやすく整形しデータ分離

In [ ]:
x_train=mnist.train.images.reshape(55000,28, 28)[:2000] # データが1次元なのでわかりやすいように2次元にする
y_train=mnist.train.labelｓ[:2000]
x_test=mnist.test.images.reshape(10000,28, 28)
y_test=mnist.test.labels

データの表示

In [ ]:
img_horizontal = x_train[2:4]
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.imshow(img_horizontal[0])
ax2.imshow(img_horizontal[1])

plt.show()

## CNNを作る

入力用placeholderを作る

In [ ]:
x_ = tf.placeholder(tf.float32, shape=[None, 28,28])
t_ = tf.placeholder(tf.float32, shape=[None,10])

一般的な画像のデータ構造の変形する   
[バッチサイズ,x,y,チャンネル]

In [ ]:
input_layer = tf.reshape(x_, [-1, 28, 28, 1])

1層目の畳込み(5×5の畳み込みフィルター×2枚)   
in   : [batch_size, 28, 28, 1]   
out  : [batch_size, 28, 28, 2]

In [ ]:
conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=2,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

2層目のmaxプーリング(サイズ２×２が２枚)   
in:[batch_size, 28, 28, 2]  
out: [batch_size, 14, 14, 2]  

In [ ]:
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

3層目の畳込み(5×5の畳み込みフィルター×2枚)   
in   : [batch_size, 14, 14, 2]   
out  : [batch_size, 14, 14, 2]

In [ ]:
conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=2,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

4層目のmaxプーリング(サイズ２×２が２枚)   
in:[batch_size, 14, 14, 2]  
out: [batch_size, 7, 7, 2]  

In [ ]:
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

１次元テンソルに変換   
in: [batch_size, 7, 7, 2]   
out:[batch_size, 7 * 7 * 2]

In [ ]:
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 2])

全結合   
in : [batch_size, 7 * 7 * 2]   
out:[batch_size, 6]  

In [ ]:
dense = tf.layers.dense(inputs=pool2_flat, units=6, activation=tf.nn.relu)

ドロップアウトと活性化

In [ ]:
dropout = tf.layers.dropout(inputs=dense, rate=0.5)
logits = tf.layers.dense(inputs=dropout, units=10)

損失関はクロスエントロピーを使う

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=t_, logits=logits))

各種オペレーション

In [ ]:
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(t_, 1))
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

学習   
バッチサイズ:20  
イテレーション: 20  

``tf.InteractiveSession()``を使うと``hoge.run()``とか``hoge.eval()``で演算ができる

In [ ]:

batch_size = 20
iteration = 20
sess = tf.InteractiveSession()

sess.run(tf.initialize_all_variables())
batch_n =  len(x_train) // batch_size
for i in range(iteration):

    x_train, y_train =shuffle(x_train, y_train)
    for j in range(batch_n-2):
        start = j * batch_size
        end = start + batch_size
        label = y_train[start:end]
        train_step.run( feed_dict={x_:x_train[start:end].astype(np.float32),t_:label.astype(np.float32)})
    start = (batch_n-1) * batch_size
    end = start + batch_size
    label = y_train[start:end]
    print("iteration: ",i)
    print("train_loss: ",cross_entropy.eval( feed_dict={x_:x_train[start:end].astype(np.float32),t_:label.astype(np.float32)}))
    print("train_acc:  ",accuracy.eval( feed_dict={x_:x_train[start:end].astype(np.float32),t_:label.astype(np.float32)}))
    print("test_loss:  ",cross_entropy.eval( feed_dict={x_:x_test.astype(np.float32),t_:y_test.astype(np.float32)}))
    print("test_acc:  ",accuracy.eval( feed_dict={x_:x_test.astype(np.float32),t_:y_test.astype(np.float32)}))
    print("----------------")

    